# Quantum Gases
### Authors: Lorenzo Braccini, Prof. Bart Hoogenboom, University College London

<font color=red>Note: the red texts are meant as side notes during the implementation.

</font> 

### Running instructions

The notebook runs from top to bottom and you should ``Run`` each input cell one after the other. 

## Introduction:

## Content of the notebook:
1. [Section 1](#1) **1D Density of States**
2. [Section 2](#2) **2D and 3D Densities of States**
3. [Section 3](#3) **Partition Functions**

In [1]:
%%capture
# IMPORTANT - run this cell to ensure videos work
!conda install -y -q ffmpeg 
!conda install jupyter_contrib_nbextensions # I am not sure if this is correct

In [2]:
# Imports

import numpy as np # advanced maths 
import matplotlib.pyplot as plt # plots 
from mpl_toolkits.mplot3d import Axes3D # 3D plots
import matplotlib.animation as animation # animation (video)
import matplotlib.gridspec as gridspec # fancy subplots
from IPython.display import HTML # shows animation in jupyter
from ipywidgets import interact, interactive, fixed, interact_manual # add interactions
from tqdm import tqdm_notebook as tqdm

In [3]:
matplotlib inline

If "save = True", the notebook outputs are saved. Note: In this case, the running time increases. 

If "gif = True", the file are saved as gif. Otherwise as mp4

In [4]:
save = True # if true, it saves.
gif = False # if true, it saves as gif. Otherwise, it saves as mp4.

## 1) <a id='1'> </a> 1D Density of States


To develop the statistical physics of a quantum gas, we need to count the quantum states of the particles. For this purpose, we consider a particle of mass $m$ in a box of length L. First, we will consider one dimentional system, and later we will expand the problem to two and three dimensions.  

### 1-D Infinite Square Well
The wavefunction of a free particle (with null potential $V(x) = 0$) has the form

$$
\psi_k \propto \sin(kx)
$$
In a 1-D infinite square well , applying boundary conditions and normalization contrain, the Schrödinger wavefunction becomes:
$$
\psi_n (x)= \sqrt{\frac{2}{L}} \sin \left(\frac{n \pi}{L}x \right)
$$
and energy:
$$
E_n = \frac{\hbar^2 k^2}{2 m} = \frac{\hbar^2 n^2 \pi^2}{2 m L^2}
$$


In [5]:
def wavefunction1D(x, n, L, t, alpha):
    """
    Calculate the wavefunction of a particle in a box 1-D
    Inputs:
    x      Values of x
    n      Quantum number
    L      Lenght of the box
    alpha  h^2 pi^2/(2 m)    
    Output: the wavefunction
    """
    return np.sin(np.pi * n * x / L)*np.cos(alpha*n**2*t/L**2)

In [6]:
%%capture

L = 1         # Lenght of the box
alpha = 1     # h^2 pi^2/(2 m)     
dt = 100      # interval ms   
Nframe = 156  # Number of frames


n_max = 4
fig1 = plt.figure(figsize = (10,10)) # initialise the figure  
ax1 = fig1.add_subplot(111)
xs_1 = np.linspace(0, L, 50) #calulate x values
colors = ['b','b','k','g','m']

ax1.set_yticks(np.arange(- 1,n_max**2 + 7, 1))
tick_y = ax1.get_yticks().tolist()
ns = np.arange(1, n_max+1, 1)**2
n = 1
for i in range(len(tick_y)):
    if tick_y[i] == 0.0:
         tick_y[i] = '0'
    elif tick_y[i] in ns:
        tick_y[i] = r'$E{{ {0} }} $'.format(n) #r'$\frac{L}{\pi}$'
        n += 1
    else:
        tick_y[i] = ''

ax1.set_xticks(np.arange(0, L+0.1, 0.1))
tick_x = ax1.get_xticks().tolist()
ns = np.arange(1, n_max, 1)**2
n = 1
for i in range(len(tick_x)):
    if tick_x[i] == 1.0:
        tick_x[i] = 'L' #r'$\frac{L}{\pi}$'
    elif tick_x[i] == 0.0:
        tick_x[i] = '0' #r'$\frac{L}{\pi}$'
    else:
        tick_x[i] = ''

def particle_box(frame):
    ax1.clear()
    t = frame*0.04 # time of the animation
    for n in range(1, n_max+1):
        ax1.plot(xs_1, n**2 + wavefunction1D(xs_1, n, L, t, alpha), color= colors[n])
        ax1.axhline(y=n**2, color='gray', alpha = 0.3)
    
    ax1.set_yticks(np.arange(- 1,n_max**2 + 7, 1))
    ax1.set_xticks(np.arange(0, L+0.1, 0.1))
    ax1.set_xlabel('x')
    ax1.set_ylabel('Energy')
    ax1.axvline(x=0, color='r')
    ax1.axvline(x=L, color='r')
    ax1.set_yticklabels(tick_y)
    ax1.set_xticklabels(tick_x)
    ax1.set_ylim(0, n_max**2 + 1.5)
    ax1.set_title('1D Infinite Square Well');

In [7]:
# create the animation
ani1 = animation.FuncAnimation(fig1, particle_box, init_func=None, interval=dt, frames = tqdm(range(Nframe))) 
HTML(ani1.to_html5_video())


In [8]:
# save the file
if save == True:
    
    # save as gif
    if gif == True:
        writergif = animation.PillowWriter(fps=10) 
        ani1.save('1D_infinite_square_well.gif', writer=writergif)
        print('The animation was saved as 1D_infinite_square_well.gif')
    
    # save as mp4
    else:
        FFwriter = animation.FFMpegWriter(fps = 10) 
        ani1.save('1D_infinite_square_well.mp4', writer = FFwriter)
        print('The animation was saved as 1D_infinite_square_well.mp4')
else:
    print('The animation was not saved')

The animation was saved as 1D_infinite_square_well.mp4


### 1D Density of States

Given the quantised nature of $k$ (and thus of the energy), the states of a particle in a box represent a series a point in the "k-space" separated by $\Delta k = \frac{\pi}{L}$. *i.e.* every increment of $\Delta k = \frac{\pi}{L}$ contains one k-state. This give the density of states:
$$
\rho (k) = \frac{\pi}{L} 
$$
Since, in 1-D, each k states correspond to an energy state $\epsilon = \frac{\hbar^2 k^2}{2 m}$, then there is one state per $\Delta \epsilon = \Delta k \cdot \frac{\Delta \epsilon}{\Delta k}$, *i.e.*:
$$
\Delta \epsilon \approx \Delta k \cdot \frac{d \epsilon}{d k} = \frac{\pi}{L} \frac{\hbar^2 k}{2 m} \propto \sqrt{\epsilon}
$$
which gives the density of states:
$$
g(\epsilon) = \frac{1}{\Delta \epsilon} \propto \frac{1}{\sqrt{\epsilon}}
$$


In [9]:
kmax_1D = 15                 # Max velocity
dk_1D = 1                    # Delta velocity
Npoints_1D = kmax_1D         # Number of lattice points

# generate lattice points 1D
rbox_1D = np.zeros((Npoints_1D,1))
count = 0
for i in range(kmax_1D): # iterate over k_x
    rbox_1D[count] = np.array([i])*dk_1D # save the position of the lattice point
    count += 1 # move to the next point

rbox_1D = np.delete(rbox_1D, 0)
Npoints_1D = len(rbox_1D)

ks_continuos_1D = np.linspace(0, kmax_1D, 50)
epsilon_continuos_1D = np.linspace(0.1, kmax_1D**2, 80)

# calculate the distance (k)
ks_1D = np.zeros(Npoints_1D)
for i in range(Npoints_1D):
    ks_1D[i] = np.linalg.norm(rbox_1D[i])
    
es_1D = ks_1D**2

# calulate the number of states for k and epsilon (discrate)
xs_k_1D = np.arange(0, kmax_1D+1 , dk_1D)
xs_e_1D = np.linspace(0, (kmax_1D)**2 , kmax_1D+1)
de_1D = xs_e_1D[1] - xs_e_1D[0]

ys_1D_k = np.zeros(len(xs_k_1D-1))
ys_1D_e = np.zeros(len(xs_e_1D-1))

for i in range(len(xs_k_1D) -1):
    ys_1D_k[i] = np.sum(np.where((ks_1D >= xs_k_1D[i] - dk_1D/2) & (ks_1D < xs_k_1D[i] + dk_1D/2) , 1, 0))

for i in range(len(xs_e_1D) - 1):
    ys_1D_e[i] = np.sum(np.where((es_1D > xs_e_1D[i] - de_1D/2) & (es_1D <= xs_e_1D[i] + de_1D/2) , 1, 0))


In [10]:
@interact(k=(0,kmax_1D - 1, dk_1D)) # interacting velocity 
def kspace(k):
    fig3 = plt.figure(figsize = (12,8),constrained_layout=True)
    grid = fig3.add_gridspec(3,2)
    
    ###### First subplot #######
    ax1 = fig3.add_subplot(grid[0,0])
    
    # plot the lattice points
    for i in range(Npoints_1D):
        if k - dk_1D/2  <= ks_1D[i] and ks_1D[i] < k + dk_1D/2 : # if they are inside the shell 
            ax1.scatter(rbox_1D[i], 0 , s = 40, color ='r', zorder=5) # plot points 
        else:
            ax1.scatter(rbox_1D[i], 0 , s = 20, color ='b', zorder=5) # plot points in different color
    
    # plot rectangle
    rectangle = plt.Rectangle((k - dk_1D/2,-2), dk_1D, 4, alpha = 0.4, color = 'r', zorder=1)
    ax1.add_artist(rectangle)
    
    # add labels, title, axes, grid 
    ax1.set_xlabel('$k$') 
    ax1.axhline(y=0, color='k', alpha = 0.8,  zorder=0)
    ax1.grid(axis = 'x')
    ax1.set_title('States in the k-space')
    ax1.set_ylim(-1,1)
    ax1.set_xlim(0, kmax_1D)
    
    # set tick axis
    ax1.set_xticks(np.arange(0,kmax_1D, 1))
    tick_k = ax1.get_xticks().tolist()
    for i in range(len(tick_k)):
        if i == 0:
            tick_k[i] = '0'
        else:
            tick_k[i] = r'$k_{{ {0} }}$'.format(i)     
    ax1.set_yticklabels([])
    ax1.set_xticklabels(tick_k)
    
    
    ###### Second subplot #######
    ax2 = fig3.add_subplot(grid[0,1])
    
    # plot number of states
    ax2.bar(xs_k_1D, ys_1D_k, width = dk_1D, edgecolor='k', facecolor = 'b', zorder=2, alpha = 0.8)
    ax2.get_children()
    ax2.get_children()[k].set_facecolor('r') 
    
    # plot theoretical 
    ax2.axhline(y=1, color='k', alpha = 0.8, zorder=3, label=r'$\rho(k) = 1$')
    
    # add labels, title, axes, grid 
    ax2.set_title('States density as function of $k$')
    ax2.set_xlabel(r'$k$')
    ax2.set_ylabel(r'$\rho(k)$')
    ax2.legend(loc = 'best', fontsize =  'large')
    ax2.set_xlim(0,kmax_1D)
    ax2.set_ylim(0,2)
    
    # set tick axis
    ax2.set_xticks(np.arange(0,kmax_1D, 1))
    ax2.set_yticklabels([])
    
    ###### Third subplot #######
    ax3 = fig3.add_subplot(grid[1,0])

    # plot the lattice points
    for i in range(Npoints_1D):
        if xs_e_1D[k] - de_1D/2  <= es_1D[i] and es_1D[i] < xs_e_1D[k] + de_1D/2 : # if they are inside the shell 
            ax3.scatter(rbox_1D[i]**2, 0 , s = 40, color ='r', zorder=5) # plot points 
        else:
            ax3.scatter(rbox_1D[i]**2, 0 , s = 20, color ='b', zorder=5) # plot points in different color
    
    # plot rectangle
    rectangle = plt.Rectangle((xs_e_1D[k] - de_1D/2,-2), de_1D, 4, alpha = 0.4, color = 'r', zorder=1)
    ax3.add_artist(rectangle)
    
    # add labels, title, axes, grid 
    ax3.set_xlabel('$\epsilon$') 
    ax3.axhline(y=0, color='k', alpha = 0.8,  zorder=0)
    ax3.grid(axis = 'x')
    ax3.set_title('States in the energy-space')
    ax3.set_ylim(-1,1)
    ax3.set_xlim(0, kmax_1D**2)
    
    # set tick axis
    ax3.set_xticks(np.arange(0,kmax_1D, 1)**2)
    tick_e_1D = ax3.get_xticks().tolist()
    for i in range(len(tick_e_1D)):
        if i == 0:
            tick_e_1D[i] = '0'
        else:
            tick_e_1D[i] = r'$E_{{ {0} }}$'.format(i)     
    ax3.set_yticklabels([])
    ax3.set_xticklabels(tick_e_1D)
    
    
    ###### Fourth subplot #######
    ax4 = fig3.add_subplot(grid[1,1])
    
    # plot number of states
    ax4.bar(xs_e_1D, ys_1D_e, width = de_1D, edgecolor='k', facecolor = 'b', zorder=2, alpha = 0.8)
    ax4.get_children()
    ax4.get_children()[k].set_facecolor('r') 

    
    # plot theoretical     
    ax4.plot(epsilon_continuos_1D, de_1D/np.sqrt(epsilon_continuos_1D), color = 'k', 
             zorder=3, label = r'$g(\epsilon) = \frac{1}{\sqrt{\epsilon}}$')
    
    # add labels, title, axes, grid 
    ax4.set_title('States density as function of $\epsilon$')
    ax4.set_xlabel(r'$\epsilon$')
    ax4.set_ylabel(r'$g(\epsilon)$')
    ax4.legend(loc = 'best', fontsize =  'large')
    ax4.set_xlim(0,kmax_1D**2)
    ax4.set_ylim(0, ys_1D_e[0]+1)
    
    # set tick axis
    ax4.set_xticklabels([])
    ax4.set_yticklabels([])
    
    ###### Fifth subplot #######
    ax5 = fig3.add_subplot(grid[2,:])
    
    # plot the wavefunction
    xs3b = np.linspace(0,L,100)
    ys3b = wavefunction1D(xs3b, k, L, 0, alpha)
    ax5.plot(xs3b, ys3b , color='b')
    
    # add labels, title, axes, grid 
    ax5.grid(axis = 'x')
    ax5.axhline(y=0, color='k', alpha = 0.8)
    ax5.axvline(x=0, color='r')
    ax5.axvline(x=L, color='r')
    ax5.set_ylim(- np.sqrt(np.pi/L)*1.05, np.sqrt(np.pi/L)*1.05)
    ax5.set_title('Wavefunction')
    ax5.set_ylabel(r'$\psi$')
    
    # set tick axis
    ax5.set_xticks(np.linspace(0,1,5))
    tick_L = ['0',  r'$\frac{L}{4}$',  r'$\frac{L}{2}$', r'$\frac{3}{4} L$','L' ]
    ax5.set_xticklabels(tick_L)
    ax5.set_yticklabels([])

interactive(children=(IntSlider(value=7, description='k', max=14), Output()), _dom_classes=('widget-interact',…

In [11]:
%%capture
############### save file ###############
if save == True:
    
    fig3 = plt.figure(figsize = (12,8),constrained_layout=True)
    grid = fig3.add_gridspec(3,2)
    ax1 = fig3.add_subplot(grid[0,0])
    ax2 = fig3.add_subplot(grid[0,1])
    ax3 = fig3.add_subplot(grid[1,0])
    ax4 = fig3.add_subplot(grid[1,1])
    ax5 = fig3.add_subplot(grid[2,:])
    
    def D1_state_denisty(k):

        ###### First subplot #######
        ax1.clear()
        # plot the lattice points
        for i in range(Npoints_1D):
            if k - dk_1D/2  <= ks_1D[i] and ks_1D[i] < k + dk_1D/2 : # if they are inside the shell 
                ax1.scatter(rbox_1D[i], 0 , s = 40, color ='r', zorder=5) # plot points 
            else:
                ax1.scatter(rbox_1D[i], 0 , s = 20, color ='b', zorder=5) # plot points in different color

        # plot rectangle
        rectangle = plt.Rectangle((k - dk_1D/2,-2), dk_1D, 4, alpha = 0.4, color = 'r', zorder=1)
        ax1.add_artist(rectangle)

        # add labels, title, axes, grid 
        ax1.set_xlabel('$k$') 
        ax1.axhline(y=0, color='k', alpha = 0.8,  zorder=0)
        ax1.grid(axis = 'x')
        ax1.set_title('States in the k-space')
        ax1.set_ylim(-1,1)
        ax1.set_xlim(0, kmax_1D)

        # set tick axis
        ax1.set_xticks(np.arange(0,kmax_1D, 1))
        tick_k = ax1.get_xticks().tolist()
        for i in range(len(tick_k)):
            if i == 0:
                tick_k[i] = '0'
            else:
                tick_k[i] = r'$k_{{ {0} }}$'.format(i)     
        ax1.set_yticklabels([])
        ax1.set_xticklabels(tick_k)


        ###### Second subplot #######
        ax2.clear()

        # %%!plot number of states
        ax2.bar(xs_k_1D, ys_1D_k, width = dk_1D, edgecolor='k', facecolor = 'b', zorder=2, alpha = 0.8)
        ax2.get_children()
        ax2.get_children()[k].set_facecolor('r') 

        # plot theoretical 
        ax2.axhline(y=1, color='k', alpha = 0.8, zorder=3, label=r'$\rho(k) = 1$')

        # add labels, title, axes, grid 
        ax2.set_title('States density as function of $k$')
        ax2.set_xlabel(r'$k$')
        ax2.set_ylabel(r'$\rho(k)$')
        ax2.legend(loc = 'best', fontsize =  'large')
        ax2.set_xlim(0,kmax_1D)
        ax2.set_ylim(0,2)

        # set tick axis
        ax2.set_xticks(np.arange(0,kmax_1D, 1))
        ax2.set_yticklabels([])

        ###### Third subplot #######
        ax3.clear()

        # plot the lattice points
        for i in range(Npoints_1D):
            if xs_e_1D[k] - de_1D/2  <= es_1D[i] and es_1D[i] < xs_e_1D[k] + de_1D/2 : # if they are inside the shell 
                ax3.scatter(rbox_1D[i]**2, 0 , s = 40, color ='r', zorder=5) # plot points 
            else:
                ax3.scatter(rbox_1D[i]**2, 0 , s = 20, color ='b', zorder=5) # plot points in different color

        # plot rectangle
        rectangle = plt.Rectangle((xs_e_1D[k] - de_1D/2,-2), de_1D, 4, alpha = 0.4, color = 'r', zorder=1)
        ax3.add_artist(rectangle)

        # add labels, title, axes, grid 
        ax3.set_xlabel('$\epsilon$') 
        ax3.axhline(y=0, color='k', alpha = 0.8,  zorder=0)
        ax3.grid(axis = 'x')
        ax3.set_title('States in the energy-space')
        ax3.set_ylim(-1,1)
        ax3.set_xlim(0, kmax_1D**2)

        # set tick axis
        ax3.set_xticks(np.arange(0,kmax_1D, 1)**2)
        tick_e_1D = ax3.get_xticks().tolist()
        for i in range(len(tick_e_1D)):
            if i == 0:
                tick_e_1D[i] = '0'
            else:
                tick_e_1D[i] = r'$E_{{ {0} }}$'.format(i)     
        ax3.set_yticklabels([])
        ax3.set_xticklabels(tick_e_1D)


        ###### Fourth subplot #######
        ax4.clear()

        # plot number of states
        ax4.bar(xs_e_1D, ys_1D_e, width = de_1D, edgecolor='k', facecolor = 'b', zorder=2, alpha = 0.8)
        ax4.get_children()
        ax4.get_children()[k].set_facecolor('r') 


        # plot theoretical     
        ax4.plot(epsilon_continuos_1D, de_1D/np.sqrt(epsilon_continuos_1D), color = 'k', 
                 zorder=3, label = r'$g(\epsilon) = \frac{1}{\sqrt{\epsilon}}$')

        # add labels, title, axes, grid 
        ax4.set_title('States density as function of $\epsilon$')
        ax4.set_xlabel(r'$\epsilon$')
        ax4.set_ylabel(r'$g(\epsilon)$')
        ax4.legend(loc = 'best', fontsize =  'large')
        ax4.set_xlim(0,kmax_1D**2)
        ax4.set_ylim(0, ys_1D_e[0]+1)

        # set tick axis
        ax4.set_xticklabels([])
        ax4.set_yticklabels([])

        ###### Fifth subplot #######
        ax5.clear()

        # plot the wavefunction
        xs3b = np.linspace(0,L,100)
        ys3b = wavefunction1D(xs3b, k, L, 0, alpha)
        ax5.plot(xs3b, ys3b , color='b')

        # add labels, title, axes, grid 
        ax5.grid(axis = 'x')
        ax5.axhline(y=0, color='k', alpha = 0.8)
        ax5.axvline(x=0, color='r')
        ax5.axvline(x=L, color='r')
        ax5.set_ylim(- np.sqrt(np.pi/L)*1.05, np.sqrt(np.pi/L)*1.05)
        ax5.set_title('Wavefunction')
        ax5.set_ylabel(r'$\psi$')

        # set tick axis
        ax5.set_xticks(np.linspace(0,1,5))
        tick_L = ['0',  r'$\frac{L}{4}$',  r'$\frac{L}{2}$', r'$\frac{3}{4} L$','L' ]
        ax5.set_xticklabels(tick_L)
        ax5.set_yticklabels([])
        
    ani2 = animation.FuncAnimation(fig3, D1_state_denisty, init_func=None, 
                                   interval=1000, frames = tqdm(range(int(kmax_1D)))) 
    
    

In [12]:
if save == True:
    # save as gif
    if gif == True:
        writergif = animation.PillowWriter(fps = 1) 
        ani2.save('1D_denisty_states.gif', writer=writergif)
        print('The animation was saved as 1D_denisty_states.gif')
    
    # save as mp4
    else:
        FFwriter = animation.FFMpegWriter(fps = 1) 
        ani2.save('1D_denisty_states.mp4', writer = FFwriter)
        print('The animation was saved as 1D_denisty_states.mp4')
else:
    print('The animation was not saved')

The animation was saved as 1D_denisty_states.mp4


In [13]:
kmax_1D = 15                 # Max velocity
dk_1D = 1                    # Delta velocity
Npoints_1D = kmax_1D         # Number of lattice points

# generate lattice points 1D
rbox_1D = np.zeros((Npoints_1D,1))
count = 0
for i in range(kmax_1D): # iterate over k_x
    rbox_1D[count] = np.array([i])*dk_1D # save the position of the lattice point
    count += 1 # move to the next point

rbox_1D = np.delete(rbox_1D, 0)
Npoints_1D = len(rbox_1D)

ks_continuos_1D = np.linspace(0, kmax_1D, 50)
epsilon_continuos_1D = np.linspace(0.1, kmax_1D**2, 80)

# calculate the distance (k)
ks_1D = np.zeros(Npoints_1D)
for i in range(Npoints_1D):
    ks_1D[i] = np.linalg.norm(rbox_1D[i])
    
es_1D = ks_1D**2

# calulate the number of states for k and epsilon (discrate)
xs_k_1D = np.arange(0, kmax_1D+1 , dk_1D)
xs_e_1D = np.linspace(0, (kmax_1D)**2 , int((kmax_1D+1)/2))

de_1D = xs_e_1D[1] - xs_e_1D[0]

ys_1D_k = np.zeros(len(xs_k_1D-1))
ys_1D_e = np.zeros(len(xs_e_1D-1))

for i in range(len(xs_k_1D) -1):
    ys_1D_k[i] = np.sum(np.where((ks_1D >= xs_k_1D[i] - dk_1D/2) & (ks_1D < xs_k_1D[i] + dk_1D/2) , 1, 0))

for i in range(len(xs_e_1D) - 1):
    ys_1D_e[i] = np.sum(np.where((es_1D > xs_e_1D[i] - de_1D/2) & (es_1D <= xs_e_1D[i] + de_1D/2) , 1, 0))


In [14]:
@interact(k=(0,kmax_1D - 1, dk_1D)) # interacting velocity 
def kspace(k):
    fig3 = plt.figure(figsize = (12,8),constrained_layout=True)
    grid = fig3.add_gridspec(3,2)
    
    ###### First subplot #######
    ax1 = fig3.add_subplot(grid[0,0])
    
    # plot the lattice points
    for i in range(Npoints_1D):
        if k - dk_1D/2  <= ks_1D[i] and ks_1D[i] < k + dk_1D/2 : # if they are inside the shell 
            ax1.scatter(rbox_1D[i], 0 , s = 40, color ='r', zorder=5) # plot points 
        else:
            ax1.scatter(rbox_1D[i], 0 , s = 20, color ='b', zorder=5) # plot points in different color
    
    # plot rectangle
    rectangle = plt.Rectangle((k - dk_1D/2,-2), dk_1D, 4, alpha = 0.4, color = 'r', zorder=1)
    ax1.add_artist(rectangle)
    
    # add labels, title, axes, grid 
    ax1.set_xlabel('$k$') 
    ax1.axhline(y=0, color='k', alpha = 0.8,  zorder=0)
    ax1.grid(axis = 'x')
    ax1.set_title('States in the k-space')
    ax1.set_ylim(-1,1)
    ax1.set_xlim(0, kmax_1D)
    
    # set tick axis
    ax1.set_xticks(np.arange(0,kmax_1D, 1))
    tick_k = ax1.get_xticks().tolist()
    for i in range(len(tick_k)):
        if i == 0:
            tick_k[i] = '0'
        else:
            tick_k[i] = r'$k_{{ {0} }}$'.format(i)     
    ax1.set_yticklabels([])
    ax1.set_xticklabels(tick_k)
    
    
    ###### Second subplot #######
    ax2 = fig3.add_subplot(grid[0,1])
    
    # plot number of states
    ax2.bar(xs_k_1D, ys_1D_k, width = dk_1D, edgecolor='k', facecolor = 'b', zorder=2, alpha = 0.8)
    ax2.get_children()
    ax2.get_children()[k].set_facecolor('r') 
    
    # plot theoretical 
    ax2.axhline(y=1, color='k', alpha = 0.8, zorder=3, label=r'$\rho(k) = 1$')
    
    # add labels, title, axes, grid 
    ax2.set_title('States density as function of $k$')
    ax2.set_xlabel(r'$k$')
    ax2.set_ylabel(r'$\rho(k)$')
    ax2.legend(loc = 'best', fontsize =  'large')
    ax2.set_xlim(0,kmax_1D)
    ax2.set_ylim(0,2)
    
    # set tick axis
    ax2.set_xticks(np.arange(0,kmax_1D, 1))
    ax2.set_yticklabels([])
    
    ###### Third subplot #######
    ax3 = fig3.add_subplot(grid[1,0])

    # plot the lattice points
    for i in range(Npoints_1D):
        if xs_e_1D[int(k/2)] - de_1D/2  <= es_1D[i] and es_1D[i] < xs_e_1D[int(k/2)] + de_1D/2 : # if they are inside the shell 
            ax3.scatter(rbox_1D[i]**2, 0 , s = 40, color ='r', zorder=5) # plot points 
        else:
            ax3.scatter(rbox_1D[i]**2, 0 , s = 20, color ='b', zorder=5) # plot points in different color
    
    # plot rectangle
    rectangle = plt.Rectangle((xs_e_1D[int(k/2)] - de_1D/2,-2), de_1D, 4, alpha = 0.4, color = 'r', zorder=1)
    ax3.add_artist(rectangle)
    
    # add labels, title, axes, grid 
    ax3.set_xlabel('$\epsilon$') 
    ax3.axhline(y=0, color='k', alpha = 0.8,  zorder=0)
    ax3.grid(axis = 'x')
    ax3.set_title('States in the energy-space')
    ax3.set_ylim(-1,1)
    ax3.set_xlim(0, kmax_1D**2)
    
    # set tick axis
    ax3.set_xticks(np.arange(0,kmax_1D, 1)**2)
    tick_e_1D = ax3.get_xticks().tolist()
    for i in range(len(tick_e_1D)):
        if i == 0:
            tick_e_1D[i] = '0'
        else:
            tick_e_1D[i] = r'$E_{{ {0} }}$'.format(i)     
    ax3.set_yticklabels([])
    ax3.set_xticklabels(tick_e_1D)
    
    
    ###### Fourth subplot #######
    ax4 = fig3.add_subplot(grid[1,1])
    
    # plot number of states
    ax4.bar(xs_e_1D, ys_1D_e, width = de_1D, edgecolor='k', facecolor = 'b', zorder=2, alpha = 0.8)
    ax4.get_children()
    ax4.get_children()[int(k/2)].set_facecolor('r') 

    
    # plot theoretical     
    ax4.plot(epsilon_continuos_1D, de_1D/np.sqrt(epsilon_continuos_1D), color = 'k', 
             zorder=3, label = r'$g(\epsilon) = \frac{1}{\sqrt{\epsilon}}$')
    
    # add labels, title, axes, grid 
    ax4.set_title('States density as function of $\epsilon$')
    ax4.set_xlabel(r'$\epsilon$')
    ax4.set_ylabel(r'$g(\epsilon)$')
    ax4.legend(loc = 'best', fontsize =  'large')
    ax4.set_xlim(0,kmax_1D**2)
    ax4.set_ylim(0, ys_1D_e[0]+1)
    
    # set tick axis
    ax4.set_xticklabels([])
    ax4.set_yticklabels([])
    
    ###### Fifth subplot #######
    ax5 = fig3.add_subplot(grid[2,:])
    
    # plot the wavefunction
    xs3b = np.linspace(0,L,100)
    ys3b = wavefunction1D(xs3b, k, L, 0, alpha)
    ax5.plot(xs3b, ys3b , color='b')
    
    # add labels, title, axes, grid 
    ax5.grid(axis = 'x')
    ax5.axhline(y=0, color='k', alpha = 0.8)
    ax5.axvline(x=0, color='r')
    ax5.axvline(x=L, color='r')
    ax5.set_ylim(- np.sqrt(np.pi/L)*1.05, np.sqrt(np.pi/L)*1.05)
    ax5.set_title('Wavefunction')
    ax5.set_ylabel(r'$\psi$')
    
    # set tick axis
    ax5.set_xticks(np.linspace(0,1,5))
    tick_L = ['0',  r'$\frac{L}{4}$',  r'$\frac{L}{2}$', r'$\frac{3}{4} L$','L' ]
    ax5.set_xticklabels(tick_L)
    ax5.set_yticklabels([])

interactive(children=(IntSlider(value=7, description='k', max=14), Output()), _dom_classes=('widget-interact',…

## 2) <a id='2'> </a> Density of States


### 2D Infinite Square Well

In a 2-D Infinite Square Well(with size $L \times L$), the Time Indipendent Schrödinger equation is separable and gives the solutions: 
$$
\psi_{n_x, n_y} (x, y)= \frac{2}{L} \sin \left(\frac{n_x \pi}{L}x \right) \sin \left(\frac{n_y \pi}{L}y \right)
$$
with energy:
$$
E_n = \frac{\hbar^2 k^2}{2 m} = \frac{\hbar^2 \pi^2}{2 m L^2} \left(n_x^2 + n_y^2 \right)
$$

In [15]:
def wavefunction2D(x,y,n_x,n_y, L):
    """
    Calculate the wavefunction of a particle in a box 2-D
    Inputs:
    x, y      Values of x and y 
    n_x, n_y  Two quantum numbers
    L         Lenght of the box
    Output: the 2d wavefunction
    """
    return np.sqrt(2/L)*np.sin(np.pi * n_x * x / L)*np.sin(np.pi * n_y * y / L)

In [16]:
L = 1 

# create points for the plot
xs = np.linspace(0, L, 50)
ys = np.linspace(0, L, 50)
xx, yy = np.meshgrid(xs,ys)

@interact(n_x=(1,5, 1), n_y =(1,5, 1)) # interacting velocity 
def infinitewell2d(n_x, n_y):
    psi3d = wavefunction2D(xx,yy,n_x,n_y, L) # calulate the wavefunction
    
    # plot the wavefunction
    fig2 = plt.figure(figsize = (10,8))
    ax3d = fig2.add_subplot(111, projection = '3d')
    ax3d.plot_surface(xx,yy,psi3d,cmap='viridis')
    
    # aditional plotting specific
    ax3d.view_init(40, 130)
    ax3d.set_xlim(0,L)
    ax3d.set_ylim(0,L)
    ax3d.set_zlim(-np.sqrt(2/L),np.sqrt(2/L))
    ax3d.set_xlabel('x')
    ax3d.set_ylabel('y')
    ax3d.set_zlabel(r'$\psi$')
    
    # set axis tick
    ax3d.set_xticks(np.arange(0, L+0.1, 0.1))
    ax3d.set_yticks(np.arange(0, L+0.1, 0.1))
    tick_x = ax1.get_xticks().tolist()
    ns = np.arange(1, n_max, 1)**2
    n = 1
    for i in range(len(tick_x)):
        if tick_x[i] == 1.0:
            tick_x[i] = 'L' #r'$\frac{L}{\pi}$'
        elif tick_x[i] == 0.0:
            tick_x[i] = '0' #r'$\frac{L}{\pi}$'
        else:
            tick_x[i] = ''
     
    ax3d.set_yticklabels(tick_x)
    ax3d.set_xticklabels(tick_x)



interactive(children=(IntSlider(value=3, description='n_x', max=5, min=1), IntSlider(value=3, description='n_y…

In [17]:
%%capture
############### save file ###############
if save == True:
    
    # plot the wavefunction
    fig2 = plt.figure(figsize = (10,8))
    ax3d = fig2.add_subplot(111, projection = '3d')
    
    def D2_infinite_square_well(frame):
        
        # calulate n_x and n_y from frame
        n_x = int(frame/4)+1
        n_y = frame%4+1

        ###### First subplot #######
        ax3d.clear()
        psi3d = wavefunction2D(xx,yy,n_x,n_y, L) # calulate the wavefunction
        ax3d.plot_surface(xx,yy,psi3d,cmap='viridis')

        # aditional plotting specific
        ax3d.view_init(40, 130)
        ax3d.set_xlim(0,L)
        ax3d.set_ylim(0,L)
        ax3d.set_zlim(-np.sqrt(2/L),np.sqrt(2/L))
        ax3d.set_xlabel('x')
        ax3d.set_ylabel('y')
        ax3d.set_zlabel(r'$\psi$')

        # set axis tick
        ax3d.set_xticks(np.arange(0, L+0.1, 0.1))
        ax3d.set_yticks(np.arange(0, L+0.1, 0.1))
        tick_x = ax1.get_xticks().tolist()
        ns = np.arange(1, n_max, 1)**2
        n = 1
        for i in range(len(tick_x)):
            if tick_x[i] == 1.0:
                tick_x[i] = 'L' #r'$\frac{L}{\pi}$'
            elif tick_x[i] == 0.0:
                tick_x[i] = '0' #r'$\frac{L}{\pi}$'
            else:
                tick_x[i] = ''

        ax3d.set_yticklabels(tick_x)
        ax3d.set_xticklabels(tick_x)
        
        
    ani3 = animation.FuncAnimation(fig2, D2_infinite_square_well, init_func=None, 
                                   interval=1000, frames = tqdm(range(int(16)))) 
    

In [18]:
if save == True:
    # save as gif
    if gif == True:
        writergif = animation.PillowWriter(fps = 1) 
        ani3.save('2D_infinite_square_well.gif', writer=writergif)
        print('The animation was saved as 2D_infinite_square_well.gif')
    
    # save as mp4
    else:
        FFwriter = animation.FFMpegWriter(fps = 1) 
        ani3.save('2D_infinite_square_well.mp4', writer = FFwriter)
        print('The animation was saved as 2D_infinite_square_well.mp4')
else:
    print('The animation was not saved')

The animation was saved as 2D_infinite_square_well.mp4


### Density of States in 2D and 3D
In 2D and 3D, we can determin $\rho (k)$ and $g(\epsilon)$ in a similar way, knowing that $\epsilon = \frac{\hbar^2 k^2}{2 m}$, with $k$ defined as $\left| \vec{k} \right| = \sqrt{k_x^2 + k_y^2}$ in 2D and $\left| \vec{k} \right| = \sqrt{k_x^2 + k_y^2 + k_z^2}$ in 3D. $\rho (k)$ is the number of states with the wave nubr between $k$ and $k + \Delta k$, and $g(\epsilon) = \rho (k) \frac{\Delta k}{\Delta \epsilon} \approx \rho (k) \frac{d k}{d \epsilon} $.
In 2D, $\rho (k)$ is the number of states which lie in the circular shell with inner radius $k$ and outer radius $k + \Delta k$, in the k-space. Similarly, in 3D, $\rho (k)$ is the number of states which lie in the spherical shell with inner radius $k$ and outer radius $k + \Delta k$, in the k-space. This implies that the densities scale with the area of the circular shell and volume of the spherical shell, for 2D and 3D space respectively.


In [19]:
kmax = 8                  # Max velocity
dk = 1                    # Delta velocity
Npoints_2D = (kmax)**2    # Number of lattice points
Npoints_3D = (kmax)**3    # Number of lattice points

# generate lattice points 2D
rbox_2D = np.zeros((Npoints_2D,2))
count = 0
for i in range(kmax): # iterate over k_x, k_y
    for j in range(kmax):
        rbox_2D[count] = np.array([i,j])*dk # save the position of the lattice point
        count += 1 # move to the next point
        
# generate lattice points 3D
rbox_3D = np.zeros((Npoints_3D,3))### a) 2-D Simulation
count = 0
for i in range(kmax): # iterate over k_x, k_y, k_z
    for j in range(kmax):
        for k in range(kmax):
            rbox_3D[count] = np.array([i,j,k])*dk # save the position of the lattice point
            count += 1 # move to the next point

# delate point with k_x = k_y (= k_z) = 0
rbox_2D = np.delete(rbox_2D, 0,0)
Npoints_2D = len(rbox_2D)
rbox_3D = np.delete(rbox_3D, 0, axis=0)
Npoints_3D = len(rbox_3D)

# generate angles for sphere
phi = np.linspace(0, np.pi/2, 20)
theta = np.linspace(0, np.pi/2, 20)
    
# find points of the unit sphere
x3d = np.outer(np.cos(phi), np.sin(theta))
y3d = np.outer(np.sin(phi), np.sin(theta))
z3d = np.outer(np.ones(np.size(phi)), np.cos(theta))

ks_continuos = np.linspace(0, kmax, 50)
epsilon_continuos = np.linspace(0, kmax**2, 50)

# calculate the distance (k)
ks_2D = np.zeros(Npoints_2D)
for i in range(Npoints_2D):
    ks_2D[i] = np.linalg.norm(rbox_2D[i])
    
ks_3D = np.zeros(Npoints_3D)
for i in range(Npoints_3D):
    ks_3D[i] = np.linalg.norm(rbox_3D[i])

es_2D = ks_2D**2
es_3D = ks_3D**2

# calulate the number of states for k and epsilon (discrate) 
xs_k = np.arange(0, kmax+1 , dk)
xs_e = np.linspace(0, (kmax)**2 , kmax+1)
de = xs_e[1] - xs_e[0]

ys_2D_k = np.zeros(len(xs_k-1))
ys_2D_e = np.zeros(len(xs_e-1))
ys_3D_k = np.zeros(len(xs_k-1))
ys_3D_e = np.zeros(len(xs_e-1))

for i in range(len(xs_k) -1):
    ys_2D_k[i] = np.sum(np.where((ks_2D >= xs_k[i] - dk/2) & (ks_2D < xs_k[i] + dk/2) , 1, 0))
    ys_3D_k[i] = np.sum(np.where((ks_3D >= xs_k[i] - dk/2) & (ks_3D < xs_k[i] + dk/2) , 1, 0))

for i in range(len(xs_e) - 1):
    ys_2D_e[i] = np.sum(np.where((es_2D > xs_e[i] - de/2) & (es_2D <= xs_e[i] + de/2) , 1, 0))
    ys_3D_e[i] = np.sum(np.where((es_3D > xs_e[i] - de/2) & (es_3D <= xs_e[i] + de/2) , 1, 0))

# xs_e += xs_e[1]

In [20]:
@interact(k=(0,kmax - 1, dk)) # interacting velocity 
def kspace(k):
    fig4 = plt.figure(figsize = (14,11))
    grid = fig4.add_gridspec(4,4,  wspace=0.3, hspace=0.5)
    
    ###### First subplot #######
    ax1 = fig4.add_subplot(grid[0:2,0:2])
    
    # plot the circles 
    if k == 0:
        circle_out = plt.Circle((0,0), dk/2, color='r', alpha = 0.4, zorder=1)     # outer circle
        circle_in = plt.Circle((0,0), 0, color='w', alpha = 1, zorder=2)            # inner circle
    else:
        circle_out = plt.Circle((0,0), k + dk/2, color='r', alpha = 0.4, zorder=1)     # outer circle
        circle_in = plt.Circle((0,0), k - dk/2, color='w', alpha = 1, zorder=2)            # inner circle
        
    rectangle_1 = plt.Rectangle((-0.48,-0.5), 0.48, kmax , alpha = 1, color = 'w', zorder=3 )
    rectangle_2 = plt.Rectangle((-0.48,-0.5), kmax , 0.5, alpha = 1, color = 'w', zorder=3 )
    ax1.add_artist(circle_out)
    ax1.add_artist(circle_in)
    ax1.add_artist(rectangle_1)
    ax1.add_artist(rectangle_2)
  
    # plot the lattice points
    for i in range(Npoints_2D):
        if k - dk/2  <= ks_2D[i] and ks_2D[i] < k + dk/2 : # if they are inside the shell 
            ax1.scatter(rbox_2D[i,0],rbox_2D[i,1], s = 30, color ='r', zorder=5) # plot points 
        else:
            ax1.scatter(rbox_2D[i,0],rbox_2D[i,1], s = 15, color ='b', zorder=5) # plot points in different color
    
    # add labels, title, axes, grid 
    ax1.set_xlabel('$k_x$') 
    ax1.set_ylabel('$k_y$')
    ax1.set_xlim( - 0.5 , kmax - 0.5)
    ax1.set_ylim(- 0.5 , kmax - 0.5)
    ax1.set_axisbelow(False)
    ax1.set_aspect(aspect = 'equal')
    ax1.axvline(x=0, color='k',zorder=4)
    ax1.axhline(y=0, color='k',zorder=4)
    ax1.set_title('2-D Lattice')
    
    ###### Second subplot #######
    ax2 = fig4.add_subplot(grid[0,2:4])
    
    # plot number of states
    ax2.bar(xs_k, ys_2D_k, width = dk, edgecolor='k', facecolor = 'b', zorder=2, alpha = 0.8)
    ax2.get_children()
    ax2.get_children()[k].set_facecolor('r') 
    
    # plot theoretical 
    ax2.plot(ks_continuos, ks_continuos*np.pi/2*dk, color = 'k', zorder=3 , label = r'$\frac{L^2}{2 \pi} \; k $')
    
    
    # add labels, title, axes, grid 
    ax2.set_title('2D density of states')
    ax2.set_xlabel(r'$k$')
    ax2.set_ylabel(r'$\rho(k)$')
    ax2.legend(loc = 'best', fontsize =  'large')
    ax2.set_xlim(0,kmax)
    
    # set tick axis
    ax2.set_xticks(np.arange(0,kmax, 1))

    ###### Third subplot #######
    ax3 = fig4.add_subplot(grid[1,2:4])
    
    # plot number of states
    ax3.bar(xs_e, ys_2D_e, width = de, edgecolor='k', facecolor = 'b', zorder=2, alpha = 0.8)
    ax3.get_children()
    ax3.get_children()[k].set_facecolor('r') 
    
    # plot theoretical 
    ax3.plot(epsilon_continuos, np.pi/4*np.ones(len(epsilon_continuos))*de, color = 'k', 
             zorder=3, label = r'$ \frac{L^2}{4 \pi} \left( \frac{2 m}{\hbar^2} \right)$') 
    
    # set tick axis
    ax3.set_xticks(np.linspace(0, (kmax)**2 , kmax+1))
    
    # add labels, title, axes, grid 
    ax3.set_xlabel(r'$\epsilon$')
    ax3.set_ylabel(r'$g(\epsilon)$')
    ax3.legend(loc = 'best', fontsize =  'large')
    ax3.set_xlim(0, (kmax)**2 )
    
    ###### Fourth subplot #######
    ax4 = fig4.add_subplot(grid[2:4,0:2], projection = '3d')
    
    # plot the two spheres
    ax4.plot_surface(k*x3d, k*y3d, k*z3d, color='r', alpha = 0.2)                # inner sphere
    ax4.plot_surface((k+dk)*x3d, (k+dk)*y3d, (k+dk)*z3d, color='r', alpha = 0.2) # outer sphere
    
    # plot the lattice points
    for i in range(Npoints_3D):
        if k  <= ks_3D[i] and ks_3D[i] < k + dk : # if they are inside the shell (-0.05 to include zero)
            ax4.scatter(rbox_3D[i,0],rbox_3D[i,1],rbox_3D[i,2], s = 20, color ='r') # plot points 
        else:
            ax4.scatter(rbox_3D[i,0],rbox_3D[i,1],rbox_3D[i,2], s = 10, color ='b') # plot points in different color
            
    # add labels, title, axes
    ax4.set_xlabel('$k_x$') 
    ax4.set_ylabel('$k_y$')
    ax4.set_zlabel('$k_z$')
    ax4.plot([rbox_3D[:,0].min() - 1, rbox_3D[:,0].max() + 1], [0,0], [0,0], color = 'black')
    ax4.plot([0, 0], [rbox_3D[:,1].min() - 1, rbox_3D[:,1].max() + 1], [0, 0], color = 'black')
    ax4.plot([0, 0], [0,0], [rbox_3D[:,2].min() - 1, rbox_3D[:,2].max() + 1], color = 'black')
    ax4.set_xlim(- 0.5 ,kmax - 0.5)
    ax4.set_ylim(- 0.5 ,kmax - 0.5)
    ax4.set_zlim(- 0.5 ,kmax -0.5)
    ax4.set_title('3-D Lattice')
    
    
    ###### Fifth subplot #######
    ax5 = fig4.add_subplot(grid[2,2:4])
    
    # plot number of states
    ax5.bar(xs_k, ys_3D_k, width = dk, edgecolor='k', facecolor = 'b', zorder=2, alpha = 0.8)
    ax5.get_children()
    ax5.get_children()[k].set_facecolor('r') 
    
    # plot theoretical 
    ax5.plot(ks_continuos, ks_continuos**2*np.pi/(2)*dk, color = 'k', zorder=3 , label = r'$\frac{L^3}{2 \pi^2} \; k^2 $') 
    
    # add labels, title, axes, grid 
    ax5.set_title('3D density of states')
    ax5.set_xlabel(r'$k$')
    ax5.set_ylabel(r'$\rho(k)$')
    ax5.legend(loc = 'best', fontsize =  'large')
    ax5.set_xticks(np.arange(0,kmax+1, 1))
    ax5.set_xlim(0,kmax)
    
    ###### Sixth subplot #######
    ax6 = fig4.add_subplot(grid[3,2:4])
    
    # plot number of states
    ax6.bar(xs_e, ys_3D_e, width = de, edgecolor='k', facecolor = 'b', zorder=2, alpha = 0.8)
    ax6.get_children()
    ax6.get_children()[k].set_facecolor('r')
    
    # plot theoretical 
    ax6.plot(epsilon_continuos, np.pi/4*np.sqrt(epsilon_continuos)*de, color = 'k', 
             zorder=3 ,label = r'$\frac{L^3}{(2 \pi)^2} \left( \frac{2 m}{\hbar^2} \right)^{3/2} \sqrt{\epsilon}$')
    
    # add labels, title, axes, grid 
    ax6.set_xticks(np.linspace(0, (kmax)**2 , kmax+1))
    ax6.set_xlabel(r'$\epsilon$')
    ax6.set_ylabel(r'$g(\epsilon)$')
    ax6.legend(loc = 'best', fontsize =  'large')
    ax6.set_xlim(0, kmax**2)

interactive(children=(IntSlider(value=3, description='k', max=7), Output()), _dom_classes=('widget-interact',)…

In [21]:
%%capture
############### save file ###############
if save == True:
    
    fig4 = plt.figure(figsize = (14,11))
    grid = fig4.add_gridspec(4,4,  wspace=0.3, hspace=0.5)
    ax1 = fig4.add_subplot(grid[0:2,0:2])
    ax2 = fig4.add_subplot(grid[0,2:4])
    ax3 = fig4.add_subplot(grid[1,2:4])
    ax4 = fig4.add_subplot(grid[2:4,0:2], projection = '3d')
    ax5 = fig4.add_subplot(grid[2,2:4])
    ax6 = fig4.add_subplot(grid[3,2:4])
    
    def D2_3_state_denisty(k):

        ###### First subplot #######
        ax1.clear()
        # plot the circles 
        if k == 0:
            circle_out = plt.Circle((0,0), dk/2, color='r', alpha = 0.4, zorder=1)     # outer circle
            circle_in = plt.Circle((0,0), 0, color='w', alpha = 1, zorder=2)            # inner circle
        else:
            circle_out = plt.Circle((0,0), k + dk/2, color='r', alpha = 0.4, zorder=1)     # outer circle
            circle_in = plt.Circle((0,0), k - dk/2, color='w', alpha = 1, zorder=2)            # inner circle

        rectangle_1 = plt.Rectangle((-0.48,-0.5), 0.48, kmax , alpha = 1, color = 'w', zorder=3 )
        rectangle_2 = plt.Rectangle((-0.48,-0.5), kmax , 0.5, alpha = 1, color = 'w', zorder=3 )
        ax1.add_artist(circle_out)
        ax1.add_artist(circle_in)
        ax1.add_artist(rectangle_1)
        ax1.add_artist(rectangle_2)

        # plot the lattice points
        for i in range(Npoints_2D):
            if k - dk/2  <= ks_2D[i] and ks_2D[i] < k + dk/2 : # if they are inside the shell 
                ax1.scatter(rbox_2D[i,0],rbox_2D[i,1], s = 30, color ='r', zorder=5) # plot points 
            else:
                ax1.scatter(rbox_2D[i,0],rbox_2D[i,1], s = 15, color ='b', zorder=5) # plot points in different color

        # add labels, title, axes, grid 
        ax1.set_xlabel('$k_x$') 
        ax1.set_ylabel('$k_y$')
        ax1.set_xlim( - 0.5 , kmax - 0.5)
        ax1.set_ylim(- 0.5 , kmax - 0.5)
        ax1.set_axisbelow(False)
        ax1.set_aspect(aspect = 'equal')
        ax1.axvline(x=0, color='k',zorder=4)
        ax1.axhline(y=0, color='k',zorder=4)
        ax1.set_title('2-D Lattice')

        ###### Second subplot #######
        ax2.clear()

        # plot number of states
        ax2.bar(xs_k, ys_2D_k, width = dk, edgecolor='k', facecolor = 'b', zorder=2, alpha = 0.8)
        ax2.get_children()
        ax2.get_children()[k].set_facecolor('r') 

        # plot theoretical 
        ax2.plot(ks_continuos, ks_continuos*np.pi/2*dk, color = 'k', zorder=3 , label = r'$\frac{L^2}{2 \pi} \; k $')

        # add labels, title, axes, grid 
        ax2.set_title('2D density of states')
        ax2.set_xlabel(r'$k$')
        ax2.set_ylabel(r'$\rho(k)$')
        ax2.legend(loc = 'best', fontsize =  'large')
        ax2.set_xlim(0,kmax)

        # set tick axis
        ax2.set_xticks(np.arange(0,kmax, 1))

        ###### Third subplot #######
        ax3.clear()

        # plot number of states
        ax3.bar(xs_e, ys_2D_e, width = de, edgecolor='k', facecolor = 'b', zorder=2, alpha = 0.8)
        ax3.get_children()
        ax3.get_children()[k].set_facecolor('r') 

        # plot theoretical 
        ax3.plot(epsilon_continuos, np.pi/4*np.ones(len(epsilon_continuos))*de, color = 'k', 
                 zorder=3, label = r'$ \frac{L^2}{4 \pi} \left( \frac{2 m}{\hbar^2} \right)$') 

        # set tick axis
        ax3.set_xticks(np.linspace(0, (kmax)**2 , kmax+1))

        # add labels, title, axes, grid 
        ax3.set_xlabel(r'$\epsilon$')
        ax3.set_ylabel(r'$g(\epsilon)$')
        ax3.legend(loc = 'best', fontsize =  'large')
        ax3.set_xlim(0, (kmax)**2 )

        ###### Fourth subplot #######
        ax4.clear()

        # plot the two spheres
        ax4.plot_surface(k*x3d, k*y3d, k*z3d, color='r', alpha = 0.2)                # inner sphere
        ax4.plot_surface((k+dk)*x3d, (k+dk)*y3d, (k+dk)*z3d, color='r', alpha = 0.2) # outer sphere

        # plot the lattice points
        for i in range(Npoints_3D):
            if k  <= ks_3D[i] and ks_3D[i] < k + dk : # if they are inside the shell (-0.05 to include zero)
                ax4.scatter(rbox_3D[i,0],rbox_3D[i,1],rbox_3D[i,2], s = 20, color ='r') # plot points 
            else:
                ax4.scatter(rbox_3D[i,0],rbox_3D[i,1],rbox_3D[i,2], s = 10, color ='b') # plot points in different color

        # add labels, title, axes
        ax4.set_xlabel('$k_x$') 
        ax4.set_ylabel('$k_y$')
        ax4.set_zlabel('$k_z$')
        ax4.plot([rbox_3D[:,0].min() - 1, rbox_3D[:,0].max() + 1], [0,0], [0,0], color = 'black')
        ax4.plot([0, 0], [rbox_3D[:,1].min() - 1, rbox_3D[:,1].max() + 1], [0, 0], color = 'black')
        ax4.plot([0, 0], [0,0], [rbox_3D[:,2].min() - 1, rbox_3D[:,2].max() + 1], color = 'black')
        ax4.set_xlim(- 0.5 ,kmax - 0.5)
        ax4.set_ylim(- 0.5 ,kmax - 0.5)
        ax4.set_zlim(- 0.5 ,kmax -0.5)
        ax4.set_title('3-D Lattice')


        ###### Fifth subplot #######
        ax5.clear()

        # plot number of states
        ax5.bar(xs_k, ys_3D_k, width = dk, edgecolor='k', facecolor = 'b', zorder=2, alpha = 0.8)
        ax5.get_children()
        ax5.get_children()[k].set_facecolor('r') 

        # plot theoretical 
        ax5.plot(ks_continuos, ks_continuos**2*np.pi/(2)*dk, color = 'k', zorder=3 , label = r'$\frac{L^3}{2 \pi^2} \; k^2 $') 

        # add labels, title, axes, grid 
        ax5.set_title('3D density of states')
        ax5.set_xlabel(r'$k$')
        ax5.set_ylabel(r'$\rho(k)$')
        ax5.legend(loc = 'best', fontsize =  'large')
        ax5.set_xticks(np.arange(0,kmax+1, 1))
        ax5.set_xlim(0,kmax)

        ###### Sixth subplot #######
        ax6.clear()

        # plot number of states
        ax6.bar(xs_e, ys_3D_e, width = de, edgecolor='k', facecolor = 'b', zorder=2, alpha = 0.8)
        ax6.get_children()
        ax6.get_children()[k].set_facecolor('r')

        # plot theoretical 
        ax6.plot(epsilon_continuos, np.pi/4*np.sqrt(epsilon_continuos)*de, color = 'k', 
                 zorder=3 ,label = r'$\frac{L^3}{(2 \pi)^2} \left( \frac{2 m}{\hbar^2} \right)^{3/2} \sqrt{\epsilon}$')

        # add labels, title, axes, grid 
        ax6.set_xticks(np.linspace(0, (kmax)**2 , kmax+1))
        ax6.set_xlabel(r'$\epsilon$')
        ax6.set_ylabel(r'$g(\epsilon)$')
        ax6.legend(loc = 'best', fontsize =  'large')
        ax6.set_xlim(0, kmax**2)
        
    ani4 = animation.FuncAnimation(fig4, D2_3_state_denisty, init_func=None, 
                                   interval=1000, frames = tqdm(range(int(kmax)))) 
    

In [22]:
if save == True:
    # save as gif
    if gif == True:
        writergif = animation.PillowWriter(fps = 1) 
        ani4.save('2D_3D_denisty_states.gif', writer=writergif)
        print('The animation was saved as 2D_3D_denisty_states.gif')
    
    # save as mp4
    else:
        FFwriter = animation.FFMpegWriter(fps = 1) 
        ani4.save('2D_3D_denisty_states.mp4', writer = FFwriter)
        print('The animation was saved as 2D_3D_denisty_states.mp4')
else:
    print('The animation was not saved')

The animation was saved as 2D_3D_denisty_states.mp4


## 3) <a id='3'> </a> Partition Function 

Once we know $g(\epsilon)$, we can approximate the partition function as: 

$$
Z_1 = \sum_{\epsilon} \Omega(\epsilon) e^{- \beta \epsilon} \approx \int_{0}^{\infty} g(\epsilon)e^{- \beta \epsilon} d \epsilon
$$
where we note that $ g(\epsilon) d \epsilon$ is the number of states with energy between $\epsilon$ and $\epsilon + d \epsilon$. Integrating for the three cases:
- 1D: $\;\;\;\;\;\; Z_{1D} = \frac{L}{2 \pi} \left( \frac{2 m}{\hbar^2} \right)^{1/2}  \int_{0}^{\infty} \frac{1}{\sqrt{\epsilon}} e^{- \beta \epsilon} d \epsilon =  L\left(\frac{2 \pi m k T}{h^2}\right)^{1/2}$


- 2D: $\;\;\;\;\;\; Z_{2D} = \frac{L^2}{4 \pi} \left( \frac{2 m}{\hbar^2} \right) \int_{0}^{\infty} e^{- \beta \epsilon} d \epsilon =  A \left(\frac{2 \pi m k T}{h^2}\right)^{2/2} $


- 3D: $\;\;\;\;\;\; Z_{3D} = \frac{L^3}{(2 \pi)^2} \left( \frac{2 m}{\hbar^2} \right)^{3/2} \int_{0}^{\infty} \sqrt{\epsilon} e^{- \beta \epsilon} d \epsilon = V \left(\frac{2 \pi m k T}{h^2}\right)^{3/2} $


In [23]:
def partition_function(T, d, m):
    """
    Calulate the partition function of dimention n as function of temperature
    Inputs:
    T       Temperature(s)
    d       Dimensions of the space
    m       Mass of the particle
    Output: Partition function 
    """
    return (m*Ts)**(d/2)

In [24]:
Tmax = 5
Ts = np.linspace(0, Tmax, 100)
m_max =10

@interact(m=(1, m_max, 1)) # interacting velocity 
def plot_distribution(m):
    fig5 = plt.figure(figsize = (10,7)) # initialise the figure and set the dimension of the plot
    ax = fig5.add_subplot(111)
    
    ax.plot(Ts, partition_function(Ts, 1, m), color = 'r', label=r'1D: $\;\; Z_{1D} = L\left(\frac{2 \pi m k T}{\hbar}\right)^{1/2}$')
    ax.plot(Ts, partition_function(Ts, 2, m), color = 'b', label=r'2D: $\;\; Z_{2D} = A\left(\frac{2 \pi m k T}{\hbar}\right)^{2/2}$')
    ax.plot(Ts, partition_function(Ts, 3, m), color = 'k', label=r'3D: $\;\; Z_{3D} = V\left(\frac{2 \pi m k T}{\hbar}\right)^{3/2}$')
    
    ax.axvline(x=0, zorder=1, color='k')
    ax.axhline(y=0, zorder=1,  color='k')
    
    ax.legend(loc='upper left')
    ax.set_xlabel('Tempertature (K)')
    ax.set_ylabel(r'$Z_1$')
    ax.set_title('Partition function of a monoatomic quantum gas')
    ax.set_xlim(-0.3, Tmax + 0.2)
    ax.set_ylim(-1, 20)
    

interactive(children=(IntSlider(value=5, description='m', max=10, min=1), Output()), _dom_classes=('widget-int…

In [25]:
%%capture
############### save file ###############
if save == True:
   
    m_max =10
    fig5 = plt.figure(figsize = (10,7)) # initialise the figure and set the dimension of the plot
    ax = fig5.add_subplot(111)
    
    def partition_animation(frame):
        
        m = (frame + 1)/20
        
        Tmax = 5
        Ts = np.linspace(0, Tmax, 100)
        ax.clear()
        ax.plot(Ts, partition_function(Ts, 1, m), color = 'r', label=r'1D: $\;\; Z_{1D} = L\left(\frac{2 \pi m k T}{\hbar}\right)^{1/2}$')
        ax.plot(Ts, partition_function(Ts, 2, m), color = 'b', label=r'2D: $\;\; Z_{2D} = A\left(\frac{2 \pi m k T}{\hbar}\right)^{2/2}$')
        ax.plot(Ts, partition_function(Ts, 3, m), color = 'k', label=r'3D: $\;\; Z_{3D} = V\left(\frac{2 \pi m k T}{\hbar}\right)^{3/2}$')

        ax.axvline(x=0, zorder=1, color='k')
        ax.axhline(y=0, zorder=1,  color='k')

        ax.legend(loc='upper left')
        ax.set_xlabel('Tempertature (K)')
        ax.set_ylabel(r'$Z_1$')
        ax.set_title('Partition function of a monoatomic quantum gas')
        ax.set_xlim(-0.3, Tmax + 0.2)
        ax.set_ylim(-1, 20)

    ani5 = animation.FuncAnimation(fig5, partition_animation, init_func=None, 
                                   interval=100, frames = tqdm(range(int((m_max - 1)*20)))) 

In [26]:
if save == True:
    # save as gif
    if gif == True:
        writergif = animation.PillowWriter(fps = 20) 
        ani5.save('partition_function.gif', writer=writergif)
        print('The animation was saved as partition_function.gif')
    
    # save as mp4
    else:
        FFwriter = animation.FFMpegWriter(fps = 20) 
        ani5.save('partition_function.mp4', writer = FFwriter)
        print('The animation was saved as partition_function.mp4')
else:
    print('The animation was not saved')

The animation was saved as partition_function.mp4


With the partition function, we can calculate the other thermodynamic variables, such as mean energy. In fact, the mean average is related to the partition function with:

$$
\left< E \right> = - \frac{\partial \ln \left( Z \right)}{\partial \beta}
$$

Following, the plots of $\ln \left( Z \right)$ and of $- \frac{\partial \ln \left( Z \right)}{\partial \beta}$ are reported as function of $\beta$. the derivative is calulate computetionaly.

In [27]:
def ln_partition_function(beta, d, m):
    """
    Calculate the partition function of dimension n as a function of temperature
    Inputs:
    T       Temperature(s)
    d       Dimensions of the space
    m       Mass of the particle
    Output: Partition function 
    """
    return np.log((m)**(d/2)/beta)

def forward_deri_part(f, beta, d, m):
    """
    Calculate the first derivative in a point using Forward Difference Method
    Inputs:
    f        Function (ln of partition function)
    T        Temperature(s)
    d        Dimensions of the space
    m        Mass of the particle
    Output:  Derivative at the points of the partition function
    """
    d_beta = beta[1] - beta[0]
    return (f(beta + d_beta, d, m) - f(beta, d, m))/(d_beta) #derivative

In [28]:
Tmax = 5
beta = np.linspace(0.01, Tmax, 100)
m_max =10

@interact(m=(1, m_max, 1)) # interacting velocity 
def plot_distribution(m):
    fig6 = plt.figure(figsize = (14,7)) # initialise the figure and set the dimension of the plot
    ax1 = fig6.add_subplot(121)
    ax2 = fig6.add_subplot(122)
    
    ax1.plot(beta, ln_partition_function(beta, 1, m), color = 'r', label='1D')
    ax1.plot(beta, ln_partition_function(beta, 2, m), color = 'b', label='2D')
    ax1.plot(beta, ln_partition_function(beta, 3, m), color = 'k', label='3D')
    
    ax1.axvline(x=0, zorder=1, color='k')
    ax1.axhline(y=0, zorder=1,  color='k')
    
    ax1.legend(loc='lower right')
    ax1.set_xlabel(r'$\beta$')
    ax1.set_ylabel(r'$\ln \left( Z_{1} \right)$')
    ax1.set_xlim(-0.3, Tmax + 0.2)
    ax1.set_ylim(-5, 5)
   
    ax2.plot(beta, - forward_deri_part(ln_partition_function, beta, 1, m), color = 'r', label='1D')
    ax2.plot(beta, - forward_deri_part(ln_partition_function, beta, 2, m), color = 'b', label='2D')
    ax2.plot(beta, - forward_deri_part(ln_partition_function, beta, 3, m), color = 'k', label='3D')
    
    ax2.axvline(x=0, zorder=1, color='k')
    ax2.axhline(y=0, zorder=1,  color='k')
    
    ax2.legend(loc='upper right')
    ax2.set_xlabel(r'$\beta$')
    ax2.set_ylabel(r'$ - \frac{\partial \ln \left( Z \right)}{\partial \beta}$')
    ax2.set_xlim(-0.3, Tmax + 0.2)
    ax2.set_ylim(-0.3, 5)
    

interactive(children=(IntSlider(value=5, description='m', max=10, min=1), Output()), _dom_classes=('widget-int…

In [29]:
%%capture
############### save file ###############
if save == True:
   
    Tmax = 5
    beta = np.linspace(0.01, Tmax, 100)
    m_max =10

    fig6 = plt.figure(figsize = (14,7)) # initialise the figure and set the dimension of the plot
    ax1 = fig6.add_subplot(121)
    ax2 = fig6.add_subplot(122)
    
    def ln_partition_animation(frame):
        
        m = (frame + 1)/20
        
        ax1.clear()
        ax2.clear()
        
        ax1.plot(beta, ln_partition_function(beta, 1, m), color = 'r', label='1D')
        ax1.plot(beta, ln_partition_function(beta, 2, m), color = 'b', label='2D')
        ax1.plot(beta, ln_partition_function(beta, 3, m), color = 'k', label='3D')

        ax1.axvline(x=0, zorder=1, color='k')
        ax1.axhline(y=0, zorder=1,  color='k')

        ax1.legend(loc='lower right')
        ax1.set_xlabel(r'$\beta$')
        ax1.set_ylabel(r'$\ln \left( Z_{1} \right)$')
        ax1.set_xlim(-0.3, Tmax + 0.2)
        ax1.set_ylim(-5, 5)

        ax2.plot(beta, - forward_deri_part(ln_partition_function, beta, 1, m), color = 'r', label='1D')
        ax2.plot(beta, - forward_deri_part(ln_partition_function, beta, 2, m), color = 'b', label='2D')
        ax2.plot(beta, - forward_deri_part(ln_partition_function, beta, 3, m), color = 'k', label='3D')

        ax2.axvline(x=0, zorder=1, color='k')
        ax2.axhline(y=0, zorder=1,  color='k')

        ax2.legend(loc='upper right')
        ax2.set_xlabel(r'$\beta$')
        ax2.set_ylabel(r'$ - \frac{\partial \ln \left( Z \right)}{\partial \beta}$')
        ax2.set_xlim(-0.3, Tmax + 0.2)
        ax2.set_ylim(-0.3, 5)
    

    ani6 = animation.FuncAnimation(fig6, ln_partition_animation, init_func=None, 
                                   interval=100, frames = tqdm(range(int((m_max - 1)*20)))) 
    
    # save as gif
    if gif == True:
        writergif = animation.PillowWriter(fps = 20) 
        ani6.save('ln_partition_function.gif', writer=writergif)
    
    
    # save as mp4
    else:
        FFwriter = animation.FFMpegWriter(fps = 20) 
        ani6.save('ln_partition_function.mp4', writer = FFwriter)

In [30]:
if save == True:
    # save as gif
    if gif == True:
        writergif = animation.PillowWriter(fps = 20) 
        ani6.save('ln_partition_function.gif', writer=writergif)
        print('The animation was saved as ln_partition_function.gif')
    
    # save as mp4
    else:
        FFwriter = animation.FFMpegWriter(fps = 20) 
        ani6.save('ln_partition_function.mp4', writer = FFwriter)
        print('The animation was saved as ln_partition_function.mp4')
else:
    print('The animation was not saved')

The animation was saved as ln_partition_function.mp4


It is possible to estraplate that the mean energy of a monoatomic gas, i.e. $ - \frac{\partial \ln \left( Z \right)}{\partial \beta}$, does not varriate with the mass. In fact, algberically calulating the derivative:
$$
\left< E \right> = \frac{d}{2} k_B T
$$
where $d$ is the dimension of the space. This is consistant with the result from equipartition theory. 

In [31]:
def monoatomic_energy(T, d):
    """
    Calculate the mean average of a monoatomic gas in d dimension.
    Inputs:
    T        Temperature
    d        Dimensions
    Output: mean energy
    """
    return d/2*T

In [32]:
Tmax = 5
Ts = np.linspace(0, Tmax, 100)
m_max =10

@interact(m=(1, m_max, 1)) # interacting velocity 
def plot_distribution(m):
    fig7 = plt.figure(figsize = (10,7)) # initialise the figure and set the dimension of the plot
    ax = fig7.add_subplot(111)
    
    ax.plot(Ts, monoatomic_energy(Ts, 1) , color = 'r', label='1D')
    ax.plot(Ts, monoatomic_energy(Ts, 2), color = 'b', label='2D')
    ax.plot(Ts, monoatomic_energy(Ts, 3), color = 'k', label='3D')
    
    ax.axvline(x=0, zorder=1, color='k')
    ax.axhline(y=0, zorder=1,  color='k')
    
    ax.legend(loc='upper right')
    ax.set_xlabel('Tempertature (K)')
    ax.set_ylabel(r'$\left< E \right>$')
    ax.set_title('Mean Energy of a monoatomic gas')
    ax.set_xlim(-0.3, Tmax + 0.2)
    ax.set_ylim(-1, 20)
    

interactive(children=(IntSlider(value=5, description='m', max=10, min=1), Output()), _dom_classes=('widget-int…

In [33]:
%%capture
if save == True:
    Tmax = 5
    Ts = np.linspace(0, Tmax, 100)
    m_max =10

    fig7 = plt.figure(figsize = (10,7)) # initialise the figure and set the dimension of the plot
    ax = fig7.add_subplot(111)
    
    ax.plot(Ts, monoatomic_energy(Ts, 1) , color = 'r', label='1D')
    ax.plot(Ts, monoatomic_energy(Ts, 2), color = 'b', label='2D')
    ax.plot(Ts, monoatomic_energy(Ts, 3), color = 'k', label='3D')
    
    ax.axvline(x=0, zorder=1, color='k')
    ax.axhline(y=0, zorder=1,  color='k')
    
    ax.legend(loc='upper right')
    ax.set_xlabel('Tempertature (K)')
    ax.set_ylabel(r'$\left< E \right>$')
    ax.set_title('Mean Energy of a monoatomic gas')
    ax.set_xlim(-0.3, Tmax + 0.2)
    ax.set_ylim(-1, 20)

In [34]:
if save == True:    
    fig7.savefig('mean_energy_of_a_monoatomic_gas.png')
    print('The plot was saved as mean_energy_of_a_monoatomic_gas.png')
else:
    print('The plot was not saved')

The plot was saved as mean_energy_of_a_monoatomic_gas.png
